# Capstone Project EDA - Auxiliary Functions 
### Author: Hugo C Marrochio
### Date: Dec 8th 2024

run on terminal 

jupyter nbconvert --to script auxiliary_functions.ipynb

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from collections import deque
import random

from scipy.optimize import minimize
from sklearn.neighbors import KernelDensity

import seaborn as sns
import plotly.graph_objects as go
import random
plt.rcParams['figure.figsize'] = (8.0, 6.0) #setting figure size

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import matplotlib.dates as mdates


from scipy.optimize import minimize
from sklearn.neighbors import KernelDensity

import seaborn as sns
import plotly.graph_objects as go
import random
plt.rcParams['figure.figsize'] = (8.0, 6.0) #setting figure size
from tensorflow.keras.callbacks import EarlyStopping

from statsmodels.tsa.statespace.sarimax import SARIMAX


from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from keras.layers import LSTM, Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from keras.initializers import HeNormal
from tensorflow.keras.callbacks import TensorBoard




from sklearn.metrics import mean_squared_error

In [ ]:
# importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.layers import GRU, Bidirectional
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import SGD



import datetime
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()


In [1]:
SEED = 1234

# Set seeds for Python, NumPy, and TensorFlow
#random.seed(SEED)
#np.random.seed(SEED)
#tf.random.set_seed(SEED)

NameError: name 'random' is not defined

## 0.a - Loading all the relevant auxiliary functions

Function to set a random seet for the notebook. For now I do not need to call it.

#Set random seed, define a function so we don't need to recall later in the notebook
def set_global_seed(seed=10):
    '''
    Set a global seed value for the notebook. If seed=10, you can just run set_global_seed()
    '''
    random.seed(seed)
    np.random.seed(seed)


Here are some auxiliary functions for plotly animation. One simply separates the data in the dataframe in order to heatmap correlation matrix. The second is an argument for the duration of the animation

In [3]:
#Define a dictionary with the data from the correlation matrices
def df_to_plotly(df):
    '''
    Simple auxiliary function breaking down the parameters of the DataFrame in order to create a plotly heatmap.

    Input:
    ------
    
    df: The Pandas DataFrame we want to plot a heatmap
    

    Output:
    ------
    z: values of df
    x: column of df
    y: index of df
    
    '''
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}
# Define frame arguments for the play button
def frame_args(duration):
    '''
    Auxiliary function for the duration of frame in the animation.

    Input:
    ------
    
    duration: Duration for a frame in the animation (I believe in miliseconds)
    

    Output:
    ------
    Dictionary with formated instructions for plotly function
    
    '''
    return {
        "frame": {"duration": duration, "redraw": True},
        "mode": "immediate",
        "fromcurrent": True,
        "transition": {"duration": duration, "easing": "linear"},
    }

This is an auxiliary function to print the correct date ranges used to calculate the correlation matrices for the plotly animation.

In [4]:
def dates_legend(df_og, row_step=15):
    '''
    Auxiliary function for heatmap animation plotting. It generates a list with dates such that shows the end date at each frame.

    Input:
    ------
    
    df_og: The Pandas DataFrame we want to analyze. Notice this is in the original orientation, where there is a `Symbol` column.
    
    row_step: What is the interval we are adding to the correlations at each frame. Default is 15.

    Output:
    ------
    list_range_dates = List of strings representing the end date for the interval being calculated at each frame. Used in the animated heatmap.
    '''
    
    #Generate just one stock for simplicity
    examp=df_og['Symbol'].iloc[0]
    df_og_ex=df_og[df_og['Symbol']==examp]

    # Let us create a list to put the correct date range in our plot
    list_range_dates=[]
    start_date=f'{df_og_ex.iloc[0,0].year}-{df_og_ex.iloc[0,0].month}-{df_og_ex.iloc[0,0].day}'
    for i in range (df_og_ex.shape[0] // row_step + 1):
        end_date=f'{df_og_ex.iloc[i*row_step,0].year}-{df_og_ex.iloc[i*row_step,0].month}-{df_og_ex.iloc[i*row_step,0].day}'
        list_range_dates.append(start_date + ' to ' + end_date)
    return list_range_dates

This function creates an animation where each frame is a correlation matrix calculated for a specific range - in the default case adding 15 data point each frame.

In [5]:
# Define function as a plotly object in order to plot the animation of Heatmap

def plot_heatmap_animation(df_og, title_plot, row_step=15):
    '''
    Auxiliary function in order to animate a heatmap in Plotly. 

    Input:
    ------

    df_og: The Pandas DataFrame we want to analyze. Notice this is in the original orientation, where there is a `Symbol` column.
    
    title_plot: A string that simply labels the animation plot. Generally want to describe the properties of the DataFrame being plotted.
    
    row_step: What is the interval we are adding to the correlations at each frame. Default is 15.

    Output:
    ------
    fig: A Plotly object, we can simply fig.show() after calling the function. We can also save it to html.
    '''
    
    # Manipulate the DataFrame such that each column corresponds to a stock. It facilitates the calculation of correlation.

    grouped_value=df_og.groupby('Symbol')['Return'].apply(list).reset_index()
    df_value=pd.DataFrame(grouped_value['Return'].tolist(), index=grouped_value['Symbol']).T


    
    # Define the row step size (e.g., every 15 trading days is the default)
    frames = []

    # Create correlation matrices for increasing number of rows
    for i in range(1, df_value.shape[0] // row_step + 1):
        corr_matrix = df_value[:i * row_step].corr()  # Compute correlation for i*row_step rows starting from 0
        frames.append(corr_matrix)
    n_frames=len(frames)
    
    #Get the list of dates, used for the legend at each frame of the animation
    list_range_dates=dates_legend(df_og,row_step)


    # Create figure
    fig = go.Figure()

    # Add traces for the initial frame
    for step in np.arange(0, n_frames, 1):
        fig.add_trace(
            go.Heatmap(df_to_plotly(frames[step]), colorscale='RdBu', name="t = " + str(step), zmin=-1, zmax=1)
        )
    
    # Ensure only the first trace is visible initially
    for trace in fig.data:
        trace.visible = False
    fig.data[0].visible = True  # First trace is visible initially



    # Create frames for the animation
    fig.frames = [
        go.Frame(data=[go.Heatmap(df_to_plotly(frames[k]), colorscale='RdBu', zmin=-1, zmax=1)],
                 name=str(k)) for k in range(n_frames)
    ]

    # Define sliders for manual frame control
    sliders = [
        {
            "pad": {"b": 10, "t": 60},
            "len": 0.9,
            "x": 0.1,
            "y": 0,
            "steps": [
                {
                    "args": [[str(k)], frame_args(0)],
                    "label": list_range_dates[k],
                    "method": "animate",
                }
                for k in range(n_frames)
            ],
        }
    ]

    # Layout with play/pause buttons and sliders
    fig.update_layout(
        title=title_plot,
        width=600,
        height=600,
        updatemenus=[{
            "buttons": [
                {
                    "args": [None, frame_args(100)],  # Play button
                    "label": "&#9654;",  # play symbol
                    "method": "animate",
                },
                {
                    "args": [[None], frame_args(0)],  # Pause button
                    "label": "&#9724;",  # pause symbol
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 70},
            "type": "buttons",
            "x": 0.1,
            "y": 0,
        }],
        sliders=sliders
    )
    return fig
    

Perform Kernel Density Estimator to the histogram "data". Adapted from Lopez Prado's book.

In [6]:
# Create the KDE from the data
def KDE_pdf(data, x_range, bandwidth=0.2):
    """
    Perform Kernel Density Estimator for the data.

    Input:
    ------
    
    data: In our case the data is the list of eigenvalues calculate from a correlation matrix.
    
    bandwidth: Parameter that control the smoothing of the KDE.

    x_range: Range in x for evaluation of the KDE
    

    Output:
    ------

    pdf: It outputs the estimated pdf function from our data

    """
    # Reshape data 
    data = data.reshape(-1, 1)
    
    # Use KernelDensity model from sklearn to fit the data
    
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(data)
    
    
    # Calculate the probability, need to exponentiate to be a pdf
    logProb = kde.score_samples(x_range)
    pdf = pd.Series(np.exp(logProb), index=x_range.flatten())
    
    return pdf

Shifted Marcenko-Pastur distribution. Shifted in the sense that the range is from (0 to $\lambda_+-\lambda_-$). 

In [7]:
def f_MP_pdf_shift(q, sigma, pts=1000):
    """
    Generate the theoretical Marchenko-Pastur PDF with shifted eigenvalues.

    Input:
    ------
    q: Aspect ratio (T/N)
    sigma: Variance (to be optimized)
    pts: Number of points for the PDF.

    Output:
    -------
    f: MP pdf as a pandas Series
    
    
    """
    lambda_max = (sigma**2) * (1 + np.sqrt(1/q))**2
    lambda_min = (sigma**2) * (1 - np.sqrt(1/q))**2
    lambda_vals = np.linspace(lambda_min + 10.**-14, lambda_max, pts)
    lambda_vals_shift=np.linspace(10.**-14, lambda_max-lambda_min, pts)
    
    # The Marchenko-Pastur PDF formula
    f = q * np.sqrt((lambda_max - lambda_vals) * (lambda_vals - lambda_min)) / (lambda_vals * 2 * np.pi * (sigma)**2)
    
    # Return as a pandas Series with lambda_vals as index
    f = pd.Series(f, index=lambda_vals_shift)
    return f

def f_MP_pdf_plot(q, sigma, pts=1000,shift=True):
    """
    Generate the theoretical Marchenko-Pastur PDF (default is with shifted eigenvalues), return both axis for plotting
    
    Input:
    ------
    q: Aspect ratio (T/N)
    sigma: Variance (to be optimized)
    pts: Number of points for the PDF.
    shift: A flag set to True. If we need to plot the MP pdf

    Output:
    -------
    f: MP pdf
    lambda_vals_shift: x range of values
    
    """
    lambda_max = (sigma**2) * (1 + np.sqrt(1/q))**2
    lambda_min = (sigma**2) * (1 - np.sqrt(1/q))**2
    lambda_vals = np.linspace(lambda_min + 10.**-14, lambda_max, pts)
    lambda_vals_shift = np.linspace( 10.**-14, lambda_max-lambda_min, pts)
    
    # The Marchenko-Pastur PDF formula
    f = q * np.sqrt((lambda_max - lambda_vals) * (lambda_vals - lambda_min)) / (lambda_vals * 2 * np.pi * (sigma)**2)

    #If flag shift is false, we return the unshifted value
    if shift==False:
        return f,lambda_vals
    
    # Return as a pandas Series with lambda_vals as index
    #f = pd.Series(f, index=lambda_vals)
    return f,lambda_vals_shift

Cost function for the minimization. We will compare the MP-pdf with the kde of the histogram of the eigenvalues.  Adapted from Lopez Prado's book.

In [8]:
def cost_function(params, eVal, bWidth,flag=False):
    """
    Cost function for the minimization between data and MP pdfs.

    Input:
    ------
    
    params: array containing both parameters being optimized, q and sigma.
    
    eval: The eigenvalue list used to calculate KDE and the data_pdf.

    bWidth: Parameter determining the "smoothing" of the KDE procedure.

    flag: Turn to True if we want to print the convergence of the sum of squared during the minimization.

    Output:
    ------

    sse: Sum of squared errors between the theoretical prediction MP_pdf and the pdf obtained by the data data_pdf.

    """

    # The parameters being optimized
    sigma = params[0]  
    q= params[1]

    
    # Generate the theoretical MP_pdf using f_MP_pdf_shift
    MP_pdf = f_MP_pdf_shift(q, sigma)
    
    # Generate data_pdf using KDE_pdf
    data_pdf = KDE_pdf(eVal, x_range=MP_pdf.index.values.reshape(-1, 1),bandwidth=bWidth)
    
    # Compute the sum of squared errors (SSE) 
    sse = np.sum((MP_pdf - data_pdf) ** 2)

    # Passing a flag, set to True if we want to print the SSE at each step
    if flag:
        print(f"SSE: {sse}")
    
    return sse

Minimization procedure to find best q and $\sigma$ to fit the kde of the correlation matrix histogram.  Adapted from Lopez Prado's book.

In [9]:
def findMaxEval(eVal, bWidth,flag=False):
    """
    Fit shifted Marcenko-Pastur pdf to the data.

    Input:
    ------
    
    eVal: Data, in our case a list of eigenvalues calculated from a correlation matrix.
    
    bWidth: Parameter determining the "smoothing" of the KDE procedure.

    flag: Turn to True if we want to print the convergence of the sum of squared during the minimization.

    Output:
    ------

    eMax: Scaled maximum random eigenvalue predicted by RMT and the MP pdf.
    
    sigma_opt = Resulting sigma from optimization. 
    
    q_opt = Resulting q from optimization. 
    """
    
    # Initial guess for the parameters being optimized
    initial_params = np.array([0.99,40])

    # Minimize cost_function defined previously. Make sure we put bounds for the parameters,
    
    result = minimize(cost_function, initial_params, args=(eVal, bWidth,flag),
                      bounds=[(1E-5, 1-1E-5),(4 + 1E-5, 50)])
    
    # Print the optimization result
    print(f"Optimization result: {result}")
    
    # If optimization succeeds, use the optimized sigma
    if result.success:
        sigma_opt = result.x[0]
        q_opt=result.x[1]
    else:
        sigma_opt = 1  # Fallback in case of failure
        q_opt=10
    
    # Calculate the maximum eigenvalue based on the optimized sigma, as described by Lopez Prado
    eMax = sigma_opt * (1 + np.sqrt(1/q_opt)) ** 2
    
    return eMax, sigma_opt, q_opt

Function to __sample__ random correlation matrices and return all the eigenvalues

In [10]:
def sample_cor_matrix(T,N,n_sample, sigma=1):
    '''
    Samples random covariance matrice, calculates the eigenvalues
    and returns them as a list.
    
    Parameters:
    ----------
    T: Dimension of the Original matrix X
    N: Dimension of the Original matrix X
    n_sample: The amouth of matrices from GOE that we will sample over
    sigma: Standard deviation of each element. If not provided, default is sigma=1

    
    
    Output:
    ------
    eig_list: List of total eigenvalues for all the sampled matrices
    '''
    eig_list=[]

    #Sample eigevanlues for different n_sample 
    for i in range(n_sample):
        # Generate random rectangular matrix
        M=sigma*np.random.randn(T, N)
        
        # Calculate correlation matrix (normalized with T for the eigenvalue distribution)
        C=np.matmul(M.T,M)/T
        
        #Calculate the eigenvalues
        eig=np.linalg.eigvalsh(C)
        eig_list.extend(eig)
    return eig_list

### Useful Denoising functions

Return Eigenvector matrix and eigenvalue such that a Hermitian Matrix decomposes as $W \Lambda W^T$

In [2]:
# return the eigenvector matrix and the eigenvalues
def eig_dec(M):
    '''
    
    Input:
    ------
    M: Symmetric matrix M
    
    Output:
    ------
    Lambda: Diagonal matrix with diagonal elements representing the eigenvalues in descending order
    
    eigVec: Matrix with eigenctors associate with the eigenvalues in Lambda
    
    '''
    eigVal, eigVec = np.linalg.eigh(M)
    
    #eigenvalues are returned in ascending order! In order to agree with the order of np.linalg.eigvals, let us reverse it

    # argsort() returns the indexes that would sort in ascending order, reverse for descending

    index=eigVal.argsort()[::-1]
    eigVal=eigVal[index]
    eigVec=eigVec[:,index]
    
    Lambda = np.diag(eigVal)

    return Lambda, eigVec

Denoise procedure. Given the list of eigenvalues, eigenvector and the number of eigenvalues that are not associated to the random range.

In [1]:
def denoise_eig(eigVal,eigVec,n_random):
    '''
    
    Input:
    ------
    
    eigVal: Eigenvalue matrix, can be for instance the output of eig_dec()

    eigVec: Eigenvector matrix, can be for instance the output of eig_dec()

    n_random: number of eigenvalues OUTSIDE the range of randomness.

    
    
    Output:
    ------

    C_norm: Symmetric matrix where the eigenvalues in the random range were regularized.
    
    '''
    eigVal_den=np.diag(eigVal).copy()
    n_total=eigVal_den.shape[0] #total size of the array of eigenvalues

    #perform the denoising for eigenvalues in the random range. Notice that descending order is important
    eigVal_den[n_random:]=eigVal_den[n_random:].sum()/(n_total-n_random)
    eigVal_den=np.diag(eigVal_den)

    #Use @ instead of np.matmult for cleaner code

    print(f'Shape of eigVal is {eigVal.shape}')
    print(f'Shape of eigVal_den is {eigVal_den.shape}')
    print(f'Shape of eigVec is {eigVec.shape}')
    
    C= eigVec @ eigVal_den @ eigVec.T

    print(f'Shape of C is {C.shape}')

    std=np.sqrt(np.diag(C))
    
    print(f'Shape of std is {std.shape}')

    print(f'Shape of outer is {np.outer(std,std).shape}')
    
    
    C_norm=C/np.outer(std,std)
    C_norm[C_norm<-1],C_norm[C_norm>1]=-1,1
    return C_norm


# Price Forecasting auxiliary functions

This function is useful in order to return scaled arrays, with respect to the training data. It is prudent to rescale the 
data in order to train a Neural Network.

In [ ]:
def data_process_array(df_train,df_test,key='Adj Close'):
    '''
    This function takes two dataframes to scale, if no other key given it assumes price 'Adj Close' is the column being analyze.

    It returns np scaled arrays between 1 and 0 with shape (n_rows,1).

    Input:
    ------
    df_train: Pandas DataFrame, the data we fit MinMaxScaler.

    df_test: Pandas DataFrame, the data we rescale based on train data's fit.
    
    key: If not specified, analyze price 'Adj Close'
    

    Output:
    -------
    scaled_array_train: Scaled train data, fit to be between 0 and 1.
    
    scaled_array_test: Scaled test array of shape (n_rows,1) using the fit of train data. 
    
    
    '''

    array_train=df_train[key].values
    array_test=df_test[key].values

    
    #Reshape

    array_train=array_train.reshape(-1,1)
    array_test=array_test.reshape(-1,1)

    #Scale the data, fit based on training

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_array_train = scaler.fit_transform(array_train)

    #Also scale the test, but only transform with scaler
    scaled_array_test = scaler.transform(array_test)

    return scaled_array_train, scaled_array_test

    

Since the prediction will be given scaled, we want to write a function to transform back to the original prices scaling.

In [ ]:
def data_process_inverse_array(df_train,array_target,key='Adj Close'):
    '''
    This function takes array_target as an input, shape (n_Array,1), and scales back to the original df_train[key] scale.
    One simple use is to scale back the NN prediction to the original prices scale.

    Input:
    ------
    df_train: Pandas DataFrame, the data we used to fit MinMaxScaler.

    array_target: Array in the range given by MinMaxScaler, use this function to rescale it back to original values.

    
    key: If not specified, analyze price 'Adj Close'
    

    Output:
    -------

    unscaled_array_target: Using the MinMaxScaler fit, take array_target and inverse back to the original scale of df_train[key].
    
    
    '''

    array_train=df_train[key].values

    
    #Reshape

    array_train=array_train.reshape(-1,1)

    #Scale the data, fit based on training

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_array_train = scaler.fit_transform(array_train)

    #Also scale the test, but only transform with scaler
    unscaled_array_target = scaler.inverse_transform(array_target)

    return unscaled_array_target

    

We want to produce smaller portions of the data timeseries to train a learning algorithm.

Consider this example of rolling window of size $5$.

Suppose a time series of interest is $[ a_1,a_2, a_3, a_4, a_5, a_6, a_7, a_8]$.

__Step 1:__

- rolling window:$X_1=[ a_1,a_2, a_3, a_4, a_5]$, 

- target is $y_1=[a_6]$.


__Step 2:__

- rolling window:$X_2=[ a_2, a_3, a_4, a_5,a_6]$, 

- Target is $y_2=[a_7]$.



__Step 3:__

- rolling window:$X_3=[ a_3, a_4, a_5, a_6, a_7]$, 

- Target is $y_3=[a_8]$.


Therefore, the feature input would be the $X_{\text{input}} = (X_1,X_2,X_3) $ and the target would be $y_{\text{input}} = (y_1,y_2,y_3)$. 
The function adds one more dimension in order to be ready for a Keras training:

- X shape: (n_input-window, window, 1)
- y shape: (n_input-window, 1)



In [ ]:
def rolling_window_time_series(array,window):
    '''
    Create a rolling window array.

    Input:
    ------

    array: Time series that we are converting into a rolling window format. Assumed to contain n_input datapoints >  window.

    window: The range to be converted into an input X, with the next point being the target y.
    

    Output:
    -------

    X: Numpy array datapoint in a rolling window format. Shape is (n_input-window, window, 1).

    y: The next point in the time series. The target for each rolling window of size window. Shape is (n_input-window, 1).
    
    
    '''
    if window>len(array):
        return print(f'Cannot create a window of size {window} bigger than the data points provided of size {len(array)}')
        
    X = []
    y = []
    for i in range(window, len(array)):
        X.append(array[i-window:i, 0])
        y.append(array[i, 0])
        
    #Transform to numpy
    X, y = np.array(X), np.array(y)

    #Reshaping for correct input for NN training
    X = np.reshape(X, (X.shape[0], X.shape[1],1))
    y = np.reshape(y, (y.shape[0],1))


    
    return X,y
    

Here we define our functions in order to train neural networks. We define separately RNN, GRU and LSTM for clarity.

We defined the default value for the best hyperparameters we encountered in our investigation, for ease of application. However, we still
passed these as input to be able investigate other hyperparameter values as needed.

We __fix a seed__ for reproducibility.

For best practices, we add Tensorboard and ModelCheckpoint in our model. Since we are not training on a huge amount of data, it is not so important for our analysis. However, I thought it would be a useful learning tool for future projects. In addition, saving the model every few epochs __is__ useful for the Reinforcement Learning algorithm.

In [ ]:
def neural_network_run_RNN(X_train, y_train, experiment,hidden_states=80, activation='ReLU',dropout=0.05,flag_optimizer=True,optimizer='Adam',learning_rate=0.001,batch_size=2, epoch=20):
    '''
    The function to train and fit neural networks! Created the base example the architecture that was mostly successful 
    for all architectures, but still put enough of hyperparameters as optional inputs so we can experiment with them. 

    Use simple Recurrent Neural Network as architecture!

    Input:
    ------
    X_train: Shape (n_input-window,window,1), the training data for the neural network.

    y_train: Shape (n_input-window,1), the value at window+1 used as a target for training.

    experiment: String describing the experiment, useful for labeling the logs. For instance, for a hidden_states experiment, call
    the f-string f'hiddenlayers_{hidden_states}'

    hidden_states: We fix the architecture as 40-hidden_states-40-1, default is hidden_states=80.

    activation: Activation function for the 40-hidden_states-40 layers, default is ReLU.

    dropout: Dropout of nodes, useful to control overfitting when it is important. Default is dropout=0.05.

    flag_optimizer: Default is True. Pass False if you want to include an optimizer with more options than learning_rate.

    optimizer: Default is Adam

    learning_rate: Learning rate for the optimizer. Default is 0.001.

    batch_size: Default is 2.

    epoch: Default is 20.
       

    Output:
    -------

    model: Keras object with the tensor network trained on (X_train,y_train).

    history: function from the Keras fit, useful for accessing the loss.
    

    
    '''

    
    print(f'Working on experiment {experiment}')

    # Fix seed for reproducibility
    
    tf.keras.utils.set_random_seed(1234)

    # Here we save the logs of the run in Tensorboard.
    
    log_dir = f"logs/fit/RNN_{experiment}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True)

    # Save in .h5 file every 10 epoch using ModelCheckpoint, function of the dimension of X_train and batch size.
    
    save_freq_ex=int(10*(X_train.shape[0])/batch_size)
    
    checkpoint_callback = ModelCheckpoint(
    filepath=f'model/RNN_{experiment}_epoch_{{epoch:02d}}.h5',  
    save_freq=save_freq_ex,                                 
    save_best_only=False,                      
    save_weights_only=False,                   
    verbose=0                                  
    )

    # Put together callbacks for EarlyStopping, TensorBoard and ModelCheckpoint
    
    callbacks = [EarlyStopping(monitor='loss', patience=4), tensorboard_callback,checkpoint_callback]

    
    # Construct the structure of the network
    
    model = Sequential()
    
    # Adding layers, fixed the first one to be the same amount as the rolling window
    
    model.add(SimpleRNN(units = 40, 
                        activation = activation,
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
    model.add(Dropout(dropout))
    
    # Middle layer, can change the size of hidden_states
    
    model.add(SimpleRNN(units = hidden_states, 
                        activation = activation,
                        return_sequences = True))

    # Fixed final layer to have same size as the first one.
    
    model.add(SimpleRNN(units = 40,activation = activation))
    
    #Output Layers
    
    model.add(Dense(units = 1,activation='ReLU')) # One prediction into the future! 


    # To compile, we are giving two different possibilities: One is that optimizer is just a string,
    # and we only change learning_rate. If we need an optimizer with more options, pass flag_optimizer=False.
    
    if flag_optimizer:
        model.compile(optimizer = eval(optimizer)(learning_rate=learning_rate), 
                        loss = "mean_squared_error")
    else:
        model.compile(optimizer = eval(optimizer), 
                        loss = "mean_squared_error")
        
        
    # Train the model!
    
    model_history=model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size,callbacks=[callbacks],verbose=0)

    # Return the model itself, as well as model_history (which contains information about loss).

    return model, model_history




In [1]:
def neural_network_run_GRU(X_train, y_train, experiment,hidden_states=80, activation='ReLU',dropout=0.05,flag_optimizer=True,optimizer='Adam',learning_rate=0.001,batch_size=2, epoch=20):
    '''
    The function to train and fit neural networks! Created the base example the architecture that was mostly successful 
    for all architectures, but still put enough o hyperparameters as optional inputs so we can experiment with them. 

    Use GRU as base architecture!


    Input:
    ------
    X_train: Shape (n_input-window,window,1), the training data for the neural network.

    y_train: Shape (n_input-window,1), the value at window+1 used as a target for training.

    experiment: String describing the experiment, useful for labeling the logs. For instance, for a hidden_states experiment, call
    the f-string f'hiddenlayers_{hidden_states}'

    hidden_states: We fix the architecture as 40-hidden_states-40-1, default is hidden_states=80.

    activation: Activation function for the 40-hidden_states-40 layers, default is ReLU.

    dropout: Dropout of nodes, useful to control overfitting when it is important. Default is dropout=0.05.

    flag_optimizer: Default is True. Pass False if you want to include an optimixer with more options than learning_rate.

    optimizer: Default is Adam

    learning_rate: Learning rate for the optimizer. Default is 0.001.

    batch_size: Default is 2.

    epoch: Default is 20.
       

    Output:
    -------

    model: Keras object with the tensor network trained on (X_train,y_train).

    history: function from the Keras fit, useful for accessing the loss.
    

    
    '''
    print(f'Working on experiment {experiment}')
    tf.keras.utils.set_random_seed(1234)

    # Save every 10 epoch, function of the dimension of X_train and batch size.
    
    log_dir = f"logs/fit/GRU_{experiment}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True)

    # Save every 10 epoch, function of the dimension of X_train and batch size.
    
    save_freq_ex=int(10*(X_train.shape[0])/batch_size)

    # Save in .h5 file every 10 epoch using ModelCheckpoint, function of the dimension of X_train and batch size.

    checkpoint_callback = ModelCheckpoint(
    filepath=f'model/GRU_{experiment}_epoch_{{epoch:02d}}.h5',  
    save_freq=save_freq_ex,
    #save_freq='epoch',                         
    #period=10,                                 
    save_best_only=False,                      
    save_weights_only=False,                   
    verbose=0                                  
    )

    # Put together callbacks for EarlyStopping, TensorBoard and ModelCheckpoint

    
    callbacks = [EarlyStopping(monitor='loss', patience=4), tensorboard_callback,checkpoint_callback]

    
    # Construct the structure of the network

    model = Sequential()
    
    # Adding layers, fixed the first one to be the same amount as the rolling window

    model.add(GRU(units = 40, 
                        activation = activation,
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
    model.add(Dropout(dropout)) 
    
    # Middle layer, can change the size of hidden_states

    model.add(GRU(units = hidden_states, 
                        activation = activation,
                        return_sequences = True))
    
    # Fixed final layer to have same size as the first one.

    
    model.add(GRU(units = 40,activation = activation))
    
    #Output Layer
    
    model.add(Dense(units = 1,activation='ReLU')) # One prediction into the future

    # To compile, we are giving two different possibilities: One is that optimizer is just a string,
    # and we only change learning_rate. If we need an optimizer with more options, pass flag_optimizer=False.

    if flag_optimizer:
        model.compile(optimizer = eval(optimizer)(learning_rate=learning_rate), # change learning rate from 0.0005 to 0.00005
                        loss = "mean_squared_error")
    else:
        model.compile(optimizer = eval(optimizer), # change learning rate from 0.0005 to 0.00005
                        loss = "mean_squared_error")
        
        
    # Train the model
    model_history=model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size,callbacks=[callbacks],verbose=0)

    # Return the model itself, as well as model_history (which contains information about loss).

    return model, model_history




The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
def neural_network_run_LSTM(X_train, y_train, experiment,hidden_states=80, activation='ReLU',dropout=0.05,flag_optimizer=True,optimizer='Adam',learning_rate=0.001,batch_size=2, epoch=20):
    '''
    The function to train and fit neural networks! Created the base example the architecture that was mostly successful 
    for all architectures, but still put enough o hyperparameters as optional inputs so we can experiment with them. 

    Use LSTM as base architecture!


    Input:
    ------
    X_train: Shape (n_input-window,window,1), the training data for the neural network.

    y_train: Shape (n_input-window,1), the value at window+1 used as a target for training.

    experiment: String describing the experiment, useful for labeling the logs. For instance, for a hidden_states experiment, call
    the f-string f'hiddenlayers_{hidden_states}'

    hidden_states: We fix the architecture as 40-hidden_states-40-1, default is hidden_states=80.

    activation: Activation function for the 40-hidden_states-40 layers, default is ReLU.

    dropout: Dropout of nodes, useful to control overfitting when it is important. Default is dropout=0.05.

    flag_optimizer: Default is True. Pass False if you want to include an optimixer with more options than learning_rate.

    optimizer: Default is Adam

    learning_rate: Learning rate for the optimizer. Default is 0.001.

    batch_size: Default is 2.

    epoch: Default is 20.
       

    Output:
    -------

    model: Keras object with the tensor network trained on (X_train,y_train).

    history: function from the Keras fit, useful for accessing the loss.
    

    
    '''
    print(f'Working on experiment {experiment}')
    tf.keras.utils.set_random_seed(1234)

    # Save every 10 epoch, function of the dimension of X_train and batch size.

    
    log_dir = f"logs/fit/LSTM_{experiment}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, write_graph=True)

    # Save every 10 epoch, function of the dimension of X_train and batch size.
    
    save_freq_ex=int(10*(X_train.shape[0])/batch_size)

    # Save in .h5 file every 10 epoch using ModelCheckpoint, function of the dimension of X_train and batch size.


    checkpoint_callback = ModelCheckpoint(
    filepath=f'model/LSTM_{experiment}_epoch_{{epoch:02d}}.h5',  
    save_freq=save_freq_ex,
    #save_freq='epoch',                         
    #period=10,                                 
    save_best_only=False,                      
    save_weights_only=False,                   
    verbose=0                                  
    )

    # Put together callbacks for EarlyStopping, TensorBoard and ModelCheckpoint

    callbacks = [EarlyStopping(monitor='loss', patience=4), tensorboard_callback,checkpoint_callback]

    
    # Construct the structure of the network

    model = Sequential()
    
    # Adding layers, fixed the first one to be the same amount as the rolling window

    model.add(LSTM(units = 40, 
                        activation = activation,
                        return_sequences = True,
                        input_shape = (X_train.shape[1],1)))
    model.add(Dropout(dropout)) 
    
    # Middle layer, can change the size of hidden_states

    model.add(LSTM(units = hidden_states, 
                        activation = activation,
                        return_sequences = True))
    
    # Fixed final layer to have same size as the first one.

    model.add(LSTM(units = 40,activation = activation))
    
    #Output Layer
    
    model.add(Dense(units = 1,activation='ReLU')) # One prediction into the future

    # To compile, we are giving two different possibilities: One is that optimizer is just a string,
    # and we only change learning_rate. If we need an optimizer with more options, pass flag_optimizer=False.

    if flag_optimizer:
        model.compile(optimizer = eval(optimizer)(learning_rate=learning_rate), # change learning rate from 0.0005 to 0.00005
                        loss = "mean_squared_error")
    else:
        model.compile(optimizer = eval(optimizer), # change learning rate from 0.0005 to 0.00005
                        loss = "mean_squared_error")
        
        
    # Train the model
    model_history=model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size,callbacks=[callbacks],verbose=0)

    # Return the model itself, as well as model_history (which contains information about loss).

    return model, model_history




## Reinforcement Learning Auxiliary Functions

First, we define a class for the ReplayBuffer. In reinforcement learning, a crucial part of the algorithm is revisiting older states, so we create a double-ended queue (deque) in order to efficiently store the states. We can efficiently modify either ends of the queue.

Due to memory limitations, we try to keep the size of the buffer between 1000-2000 entries.

Let us describe the important methods of the class:

-  `__init__`: Create a deque with default size 2000 (we will create a smaller one in the main notebook).
  
-  `store`: Adds experience to the deque. For instance, experience can be a tuple of information about the transition the agent is taking,
so say experience = (state, action, reward, next_state). If the list is full, older experiences will be deleted.

-  `sample`: Sample a random batch of experiences in the deque.

-  `size`: Returns how many experiences are stored.

  

In [1]:
class ReplayBuffer:

    '''
    ReplayBuffer for storing and sampling experiences in reinforcement learning.

    The ReplayBuffer is a double-ended queue (deque) used to efficiently store experiences of the agent 
    interacting with the environment. This enables revisiting older states to improve the stability of learning.

    Input:
    ------

    max_size: The maximum number of experiences the buffer can hold. Defaults to 2000. 
              Older experiences are automatically removed when the buffer exceeds this size.

    Methods:
    --------

    store: Adds an experience to the buffer. Experience is typically a tuple (state, action, reward, next_state).
           If the buffer is full, the oldest experience is removed to make space.

    sample: Returns a random batch of experiences from the buffer. The size of the batch is specified by the user.
            This helps diversify training by exposing the agent to a variety of transitions.

    size: Returns the current number of experiences stored in the buffer.

    Output:
    -------

    None: The class itself does not return anything but provides methods to store and retrieve experiences. 
    
    '''


    
    def __init__(self, max_size=2000,seed=1234):
        self.buffer = deque(maxlen=max_size)
        #self.seed = seed
        #random.seed(seed)

    
    def store(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        #random.seed(self.seed)
        return random.sample(self.buffer, batch_size)
    
    def size(self):
        return len(self.buffer)

Next, we create a Q-network for reinforcement learning. Due to computational constraints, we do not create a very deep network, but it still enough to demonstrate the capabilities of the model.

We essentially create a Keras Simple RNN network with

- SimpleRNN with 16 units to process sequential data.
- Dropout layer (rate 0.1) to reduce overfitting.
- Dense layer with 8 units and ReLU activation.
- Dense output layer with 3 units (actions, __Buy, Hold, Sell__) and linear activation.
- Adam optimizer and MSE loss.

In [ ]:
def create_q_network(window_size):
    '''
    Create a Q-Network for reinforcement learning.

    This function builds a neural network using Keras to approximate the Q-function, 
    which predicts the expected rewards for actions (Buy, Hold, Sell) based on input features.
    We pass a seed for reproducibility.
    
    The hyperparameters of the network is:
        - SimpleRNN with 16 units to process sequential data.
        - Dropout layer (rate 0.1) to reduce overfitting.
        - Dense layer with 8 units and ReLU activation.
        - Dense output layer with 3 units (actions, __Buy, Hold, Sell__) and linear activation.
        - Adam optimizer and MSE loss.

    Input:
    ------

    window_size: Number of past time steps used as input. The total input size is window_size + 1, 
                 including the predicted price.

    Output:
    -------

    model: A compiled Keras model.

    '''

    # Seed for reproducibility
    tf.keras.utils.set_random_seed(1234)
    
    input_features = window_size + 1  # Include predicted price as one of the input!

    # Create the NN architecture
    
    model = Sequential([
        SimpleRNN(16, input_shape=(input_features, 1), return_sequences=False),
        Dropout(0.1),
        
        # Next layers
        Dense(8, activation='relu'),
        Dense(3, activation='linear')  # 3 actions: Buy, Hold, Sell
    ])
    model.compile(optimizer=Adam(learning_rate=0.001)
, loss='mse')
    
    return model

Next, we either take an exploration action, which means a random experience, or we exploit the best action we have (based on Q-value). 

We define the following function, that takes epsilon to control the probability of choosing exploration (1-epsilon for exploitation). 

In [ ]:
def choose_action(q_values, epsilon, seed=SEED):
    '''
    Choose an action using an epsilon-greedy strategy.

    This function selects an action based on Q-values, balancing exploration and exploitation,
    a key aspect of Q-learning.

    Epsilon controls the probability it chooses a random exploring action. Otherwise, it selects
    a greedy action (controlled by highest Q-value).
   

    Input:
    ------

    q_values: A list or array of Q-values for each action.

    epsilon: A float (0 <= epsilon <= 1) representing the exploration probability. High epsilon favors exploration.
           

    Output:
    -------

    action: An integer representing the chosen action:
            - Random action (if exploring) or
            - The index of the maximum Q-value (if exploiting).
    '''
    #np.random.seed(seed)

    if np.random.rand() < epsilon:
        return np.random.randint(3)  # Random action, 3 since we have 3 possible actions
    return np.argmax(q_values)  # Greedy action


In reinforcement learning we want to stabilize training by keeping the target network fixed for a few steps while 
updating the Q-network. The next function updates the weights of the target when called.

In [ ]:
def update_target_network(q_network, target_network):
    '''
    Update the target network with the weights of the Q-network.

    This function synchronizes the target network by copying the weights 
    from the Q-network. 
    
    In reinforcement learning we want to stabilize 
    training by keeping the target network fixed for a few steps while 
    updating the Q-network.

    Input:
    ------

    q_network: The main Keras Q-network model whose weights are being updated frequently.

    target_network: The target Keras model to be updated with the weights of the Q-network after a few steps.

    Output:
    -------

    None: The target network is updated in place.
    
    '''

    target_network.set_weights(q_network.get_weights())

The next function calculates the reward based on portfolio value change and market conditions. This is the heart of the learning algorithms and by any means we provide the definitive ideal reward function. We tried to balance between instant reward of making a profit every day with finding incentives/penalties given the market conditions.

There are many possible fine-tuning for the reward function, we populated as default the values relevant for our example. 

The relevant inputs are

- portfolio: Current portfolio value before taking the action.

- positions: Current number of units of stock held.

- current_price: The current price of the stock.

- action: Integer. The action taken by the agent:
            - 0: Buy
            - 1: Hold
            - 2: Sell

- aux_mean: Auxiliary mean, the mean of the previous prices the agent has access to.

- aux_std: Auxiliary standard deviation, the std of the previous prices the agent has access to.

- scaling_default: (default=0.05). Minimum value for the scaling factor applied to additional incentives and penalties. Generally we want a scale to be the order of the typical daily reward.

- incentive_price_mean: (default=1.5). Multiplier for the reward/penalty adjustment based on price deviation from the auxiliary mean.

- sell_incentive: (default=1.7). Multiplier to incentive when the agent sells at peak prices.

- hold_stable: (default=1.2). Multiplier for rewarding the agent for holding in stable price ranges.

- hold_extreme: (default=0.75). Multiplier for penalizing the agent for holding during extreme price deviations.


In [ ]:
def calculate_reward(portfolio, positions, current_price, action, aux_mean, aux_std, augmented_state, \
                     scaling_default=0.1,incentive_price_mean=2,sell_incentive=1,hold_stable=0.8,hold_extreme=1):
    '''
    Calculate the reward based on portfolio value change and market conditions.

    This is the heart of the learning algorithms and by any means the definitive ideal reward function.
    We tried to balance between instant reward of making a profit every day with finding incentives/penalties given the market conditions.

    There are many possible fine-tuning for the reward function, we populated as default the values relevant for our example. 
    


    Input:
    ------

    portfolio: Current portfolio value before taking the action.

    positions: Current number of units of stock held.

    current_price: The current price of the stock.

    action: Integer. The action taken by the agent:
            - 0: Buy
            - 1: Hold
            - 2: Sell

    aux_mean: Auxiliary mean, the mean of the previous prices the agent has access to.

    aux_std: Auxiliary standard deviation, the std of the previous prices the agent has access to.

    augmented_state: 

    scaling_default: (default=0.05). Minimum value for the scaling factor applied to additional incentives and penalties. Generally we want a scale
    to be the order of the typical daily reward.

    incentive_price_mean: (default=1.5). Multiplier for the reward/penalty adjustment based on price deviation from the auxiliary mean.

    sell_incentive: (default=1.7). Multiplier to incentive when the agent sells at peak prices.

    hold_stable: (default=1.2). Multiplier for rewarding the agent for holding in stable price ranges.

    hold_extreme: (default=0.75). Multiplier for penalizing the agent for holding during extreme price deviations.

    Output:
    -------

    reward: The calculated reward, which incorporates portfolio value change and adjustments based on the action and market conditions.


    '''
    # Calculate portfolio value change
    portfolio_new = portfolio + positions * current_price
    raw_reward = portfolio_new - portfolio  # Absolute change

    # Proportional reward
    reward = (raw_reward / portfolio) if portfolio != 0 else 0


    # Scaling factor for additional incentives/penalties
    scaling_factor = max(scaling_default, abs(reward))


    
    # Adjust reward based on price deviation from mean
    price_deviation = (aux_mean - current_price) / aux_std
    reward += scaling_factor * incentive_price_mean * price_deviation


    # Action-specific incentives
    if action == 2:  # Sell
        # Extra incentive for selling at peaks
        reward += scaling_factor * sell_incentive * max(0, -price_deviation)
        

    elif action == 1:  # Hold
        # Bonus for holding in a stable price range
        holding_bonus = max(0, 1 - abs(price_deviation))
        reward += scaling_factor * hold_stable * holding_bonus

        

        # Penalty for holding during extreme deviations
        reward -= scaling_factor * hold_extreme * abs(price_deviation)


    elif action == 0:  # Buy
        # Incentive for buying during a price dip
        momentum = current_price - aux_mean
        if momentum < 0:  # Price is below mean
            reward += scaling_factor * 1.4

            


    # Final condition to make sure we don't simply sell all our positions!
    
    if positions==0 and action!=0:
        reward -= scaling_factor*2
        
        
    return reward



The following is an auxiliary function to train the Q-network. Since `q_network` is a Keras object, it updates its weights in-place, so we do not need to return it.

This function samples a batch of past experiences from the replay buffer (custom class we defined earlier) and uses Keras' NN as a function approximator of the Bellman equation. The replay buffer ensures the network learns from diverse experiences.

The Q-network uses these experiences to estimate the rewards (Q-values) for different actions. The target network provides stable Q-value targets to prevent oscillations during training. We update the weights of the target network every few episodes.

If the Q-network is well-trained, it should accurately approximate the rewards of different actions, enabling optimized decision-making during testing or real-world scenarios.

In [ ]:
def train_q_network(q_network, target_network, replay_buffer, batch_size, gamma, seed=SEED):
    """
    Train the Q-network using a batch of experiences from the replay buffer.

    Input:
    ------
    q_network: The current Q-network to be trained. (keras.Model)
    
    target_network: The target Q-network for stability in training. (keras.Model)
    
    replay_buffer: Buffer storing past experiences (state, action, reward, next state). (Custom ReplayBuffer class)
    
    batch_size: The number of experiences to sample for training.
    
    gamma: Discount factor for future rewards. Parameter in Bellman equation.

    Output:
    -------
    
    list: Updated Q-values for the first sample in the batch.
        
    loss: Training loss for the batch.
    
    """
    #np.random.seed(seed)  
    #random.seed(seed)  

    if replay_buffer.size() < batch_size:
        return None, None  # Not enough samples to train

    # Sample a batch from the replay buffer
    batch = replay_buffer.sample(batch_size)
    states, actions, rewards, next_states = zip(*batch) #zip to "unzip" the tuples return by the replay_buffer.sample

    # Prepare arrays for training
    states = np.array(states)  # Shape: (batch_size, state_dim)
    next_states = np.array(next_states)  # Shape: (batch_size, state_dim)

    # Predict Q-values for current states and next states
    q_values = q_network.predict(states, verbose=0)
    next_q_values = target_network.predict(next_states, verbose=0)

    # Update Q-values using the Bellman equation
    for i in range(batch_size):
        q_values[i, actions[i]] = rewards[i] + gamma * np.max(next_q_values[i])

    # Train the Q-network
    history = q_network.fit(states, q_values, verbose=0)
    loss = history.history['loss'][0]

    return q_values[0].tolist(), loss


Since training the Reinforcement Learning model can crash the kernel (specially for more data and longer rolling window), it is important to keep a very detailed log of the episode under consideration.

We save the q-network as an h5 file, record the parameters we are at the end of the training, record information about epsilon (as we decrease over episodes in order to change the balance between exploration and exploitation), and also the buffer of states we have visited.

If the kernel crashes, we can use this information to resume training as intended.

In [2]:
def log_episode_data(episode, q_network, replay_buffer, metrics_df, reward, total_reward, \
                     portfolio, positions, epsilon, epsilon_min, epsilon_decay):
    """
    Log metrics, save the Q-network, and export the replay buffer for a given episode.

    Input:
    ------
    episode: The current episode number. 
    
    q_network: The Q-network to save. (keras.Model)
    
    replay_buffer: Buffer storing past experiences. (Custom ReplayBuffer class)
    
    metrics_df: DataFrame to store episode metrics. (pandas.DataFrame)
    
    reward: Reward for the current step. 
    
    total_reward: Accumulated reward for the episode. 
    
    portfolio: Portfolio value at the end of the episode. 
    
    positions: Number of stocks held at the end of the episode. 
    
    epsilon: Current epsilon value for exploration. 
    
    epsilon_min: Minimum epsilon value for exploration. 
    
    epsilon_decay: Epsilon decay rate per episode. 

    Output:
    -------
    metrics_df: Updated DataFrame containing metrics for all episodes.
    
    """

    # Save Q-network checkpoint
    q_network.save(f'2trading_q_network_episode_{episode+1}.h5')

    # Save metrics as a new row of a data frame, then concat with the previous log.
    new_row = pd.DataFrame([{
        'episode': episode,
        'reward': reward,
        'total_reward': total_reward,
        'portfolio': portfolio,
        'positions': positions,
        'epsilon': epsilon,
        'epsilon_min': epsilon_min,
        'epsilon_decay': epsilon_decay
    }])
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)
    metrics_df.to_csv('metrics_df.csv', index=False)

    # Save replay buffer
    buffer_df = pd.DataFrame(list(replay_buffer.buffer), columns=['augmented_state', 'action', 'reward', 'next_augmented_state'])
    buffer_df.to_csv('replay_buffer.csv', index=False)

    return metrics_df
